In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
stop = list(set(stopwords.words('english')))

In [2]:
f_train = open('../data/train_14k_split_conll.txt','r',encoding='utf8')
line_train = f_train.readlines()

f_val = open('../data/dev_3k_split_conll.txt','r',encoding='utf8')
line_val = f_val.readlines()

f_test = open('../data/Hindi_test_unalbelled_conll_updated.txt','r',encoding='utf8')
line_test = f_test.readlines()

In [3]:
def get_data(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_doc_sentiment = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip().lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                sentiment = line.split('\t')[2]
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_doc_sentiment.append(sentiment)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    train_df['sentiment'] = train_doc_sentiment
    
    return train_df

In [4]:
def get_data_test(lines):
    doc_count = 0
    i = 0
    #global train_word, train_word_type, train_doc_sentiment, train_uid, train_doc_id
    train_word = []
    train_word_type = []
    train_uid = []
    train_doc_id = []

    while i < len(lines):
        '''
        if i < len(lines) and lines[i].split('\t')[0] in ['http','https']:
            i += 1
            while i < len(lines) and lines[i].split('\t')[0] != '/':
                i += 1
                 
            i += 1    
        ''' 

        if i < len(lines) and lines[i].split('\t')[0] in ['@']:
            i += 2

        if i < len(lines):
            line = lines[i]
            line = line.replace('\n','').strip().lower()
            if line.split('\t')[0] == 'meta':
                doc_count += 1
                uid = int(line.split('\t')[1])
                #if doc_count == 1575:
                #    print (uid)
                i += 1

            elif len(line.split('\t')) >= 2:
                if line.split('\t')[0] not in stop:
                    train_uid.append(uid)
                    train_word.append(line.split('\t')[0])
                    train_word_type.append(line.split('\t')[1])
                    train_doc_id.append(doc_count)

                i += 1

            else:
                i += 1

    train_df = pd.DataFrame()
    train_df['doc_id'] = train_doc_id
    train_df['word'] = train_word
    train_df['word_type'] = train_word_type
    train_df['uid'] = train_uid
    
    return train_df

In [5]:
train_df = get_data(line_train)
val_df = get_data(line_val)
test_df = get_data_test(line_test)

In [6]:
train_df.shape, val_df.shape, test_df.shape

((289253, 5), (62618, 5), (61735, 4))

In [7]:
train_df.uid.nunique(), val_df.uid.nunique(), test_df.uid.nunique()

(14000, 3000, 3000)

In [8]:
train_df.tail(5)

,doc_id,word,word_type,uid,sentiment
289248,14000,//,o,3308,neutral
289249,14000,.,o,3308,neutral
289250,14000,co,eng,3308,neutral
289251,14000,/,o,3308,neutral
289252,14000,cs3vtzop3q,eng,3308,neutral


In [9]:
train_df = train_df[train_df.word != 'http']
train_df = train_df[train_df.word != 'https']
train_df = train_df[train_df.word_type != 'o']
print (train_df.shape)

val_df = val_df[val_df.word != 'http']
val_df = val_df[val_df.word != 'https']
val_df = val_df[val_df.word_type != 'o']
print (val_df.shape)

test_df = test_df[test_df.word != 'http']
test_df = test_df[test_df.word != 'https']
test_df = test_df[test_df.word_type != 'o']
print (test_df.shape)

(223215, 5)
(48339, 5)
(47683, 4)


In [10]:
train_df.word = train_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
train_df = train_df[train_df.word.str.len() >= 3]
print (train_df.shape)

val_df.word = val_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
val_df = val_df[val_df.word.str.len() >= 3]
print (val_df.shape)

test_df.word = test_df.word.apply(lambda x: re.sub("[^a-zA-Z0-9]", "",x))
test_df = test_df[test_df.word.str.len() >= 3]
print (test_df.shape)

(175903, 5)
(38082, 5)
(37583, 4)


In [11]:
train_df.groupby(['uid'])['word'].count().describe()

count    14000.000000
mean        12.564500
std          3.873212
min          1.000000
25%         10.000000
50%         13.000000
75%         15.000000
max         25.000000
Name: word, dtype: float64

In [12]:
val_df.groupby(['uid'])['word'].count().describe()

count    3000.000000
mean       12.694000
std         3.839177
min         2.000000
25%        10.000000
50%        13.000000
75%        15.000000
max        24.000000
Name: word, dtype: float64

In [13]:
test_df.groupby(['uid'])['word'].count().describe()

count    3000.000000
mean       12.527667
std         3.855756
min         2.000000
25%        10.000000
50%        13.000000
75%        15.000000
max        23.000000
Name: word, dtype: float64

In [14]:
train_texts = train_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index()
train_texts = pd.merge(train_texts,train_df[['uid','sentiment']],how='left').drop_duplicates().reset_index(drop=True)
train_texts.columns = ['uid','text','sentiment']

val_texts = val_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
val_texts = pd.merge(val_texts,val_df[['uid','sentiment']],how='left').drop_duplicates()
val_texts.columns = ['uid','text','sentiment']

test_texts = test_df.groupby(['uid'],sort=True)['word'].apply(lambda x: " ".join(x)).reset_index().reset_index(drop=True)
test_texts.columns = ['uid','text']

pd.concat([train_texts[['text']],val_texts[['text']],test_texts[['text']]], axis=0).to_csv('../data/alltexts.txt',index=False,header=False)

In [15]:
! ../models/fastText-0.2.0/fasttext skipgram -input ../data/alltexts.txt -output ../models/fasttext -minn 2 -maxn 20 -dim 200 -lr .1 -minCount 0


Read 0M words
Number of words:  51516
Number of labels: 0
Progress: 100.0% words/sec/thread:    9929 lr:  0.000000 loss:  2.133549 ETA:   0h 0m


In [16]:
#! echo "modi" | ../models/fastText-0.2.0/fasttext print-word-vectors ../models/fasttext.bin 

In [17]:
train_texts.shape, val_texts.shape, test_texts.shape

((14000, 3), (3000, 3), (3000, 2))

In [18]:
train_texts.head(10)

,uid,text,sentiment
0,8,love looks good maddie ako lang yung sobrang m...,neutral
1,14,min lyching manakgupta mein kahna nae chahta q...,neutral
2,26,best luck sir world cup liye bhot bhot subhkam...,positive
3,27,yes great dialogues one also chupke chupke chh...,positive
4,33,tarekfatah tere baap liye bola kya tha bhadwe ...,negative
5,38,desh bhakti baat wahi samajh sakte hai khud de...,negative
6,41,madarchod mulle mathura nahi dikha tha jab mul...,negative
7,48,manya pradhan mantri mahoday shriman narendra ...,positive
8,56,nsharif kiya tum apne baap oulad nahi kyun usy...,negative
9,59,youtube rha nhi modi phir ban gya chupchaap aa...,neutral


In [19]:
val_texts.head(10)

,uid,text,sentiment
0,3,pakistan ghra tauq pakistan israel tasleem nah...,negative
13,12,gonna start another june sour note uhhhh yes y...,neutral
23,23,caring bohot jyada caring courier wale bsdk si...,negative
35,24,sarfaraza nonesense kabhi baymani per bani tea...,positive
49,45,pakistani team effort aagey allah marziiiiiiii...,positive
59,61,ankita shah8 bhadve photo elections pehle jyad...,negative
69,64,krishna jcb full trend chal rahi,positive
75,77,sharma1 kavita sharma4 sunita kal jab evm seal...,negative
90,81,accha kiya invite nai kiya corrupted party ind...,negative
103,101,nolo weni ankere gae weekend,positive


In [20]:
test_texts.head(10)

,uid,text
0,1,keh aese rahe jaise pakistan wale karte south ...
1,4,anus prerna way ran saving ram ram jay rajaraa...
2,17,shukar hai pathan nae warna ptm nay wae dramay...
3,18,harsh pen decision much option arm big second ...
4,34,keep saying kenyan rugby beautiful save kru dr...
5,43,guru give bless sewa simrn parmarth good deeds...
6,136,tum logo zindagi mai kabhi india map nahi dhek...
7,139,itne gandi priye aapke profile pic modi lga le...
8,158,google translate working aisi jagah net aata
9,162,qaira sahb great man apka beta nhayat nfees na...


In [21]:
all_words = set(pd.concat([train_df[['word']],val_df[['word']],test_df[['word']]], axis=0).word)
print ("Total number of words {}".format(len(all_words)))

Total number of words 51515


In [22]:
import io
from tqdm import tqdm

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        if tokens[0] in all_words:
            data[tokens[0]] = list(map(float, tokens[1:]))
    return data

In [23]:
embeddings_index1 = load_vectors('../models/fasttext.vec')

51516it [00:02, 21990.61it/s]


In [24]:
embeddings_index2 = load_vectors('../models/wiki-news-300d-1M-subword.vec')

999994it [00:20, 49257.39it/s]


In [25]:
embed_dim1 = 200
embed_dim2 = 300

In [48]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Conv1D, Input, Concatenate, Dropout
from keras.regularizers import l1, l2
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import load_model
from keras import initializers
import keras.backend as K
from keras.engine import Layer

In [27]:
all_words = pd.concat([train_df[['word']],val_df[['word']],test_df[['word']]], axis=0)

In [28]:
all_words = all_words.word.value_counts().reset_index()
all_words.columns = ['word','tot_count']

In [29]:
all_words[all_words.tot_count >= 2].shape

(16771, 2)

In [30]:
max_fatures = 16000
max_len = 20

In [31]:
all_texts = pd.concat([train_texts[['uid','text','sentiment']],val_texts[['uid','text','sentiment']]],axis=0)

In [32]:
all_texts.sentiment.value_counts()

neutral     6392
positive    5616
negative    4992
Name: sentiment, dtype: int64

In [33]:
lstm_out = 128 #dimension of the lstm output 
n_output = all_texts.sentiment.nunique() #number of possible outputs

In [34]:
senti_dict = {'negative':0,'neutral':1,'positive':2}
in_senti_dict = {0:'negative',1:'neutral',2:'positive'}

In [35]:
all_texts.sentiment = all_texts.sentiment.apply(lambda x: senti_dict[x])

In [36]:
tokenizer = Tokenizer(oov_token='UNK', num_words=max_fatures+1)
tokenizer.fit_on_texts(all_texts.text.values.tolist())
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= max_fatures}
#tokenizer.word_index[tokenizer.oov_token] = max_fatures + 1

In [37]:
X = tokenizer.texts_to_sequences(all_texts.text.values)
X = pad_sequences(X,maxlen=max_len,padding="post")
y = pd.get_dummies(all_texts.sentiment).values

testX = tokenizer.texts_to_sequences(test_texts.text.values)
testX = pad_sequences(testX,maxlen=max_len,padding="post")

In [38]:
word_index = tokenizer.word_index
print (len(word_index),X.shape, testX.shape)

16000 (17000, 20) (3000, 20)


In [39]:
tokenizer.word_index

{'UNK': 1,
 'hai': 2,
 'nahi': 3,
 'bhi': 4,
 'aur': 5,
 'modi': 6,
 'aap': 7,
 'sir': 8,
 'koi': 9,
 'nhi': 10,
 'love': 11,
 'bjp': 12,
 'hain': 13,
 'good': 14,
 'kar': 15,
 'kya': 16,
 'desh': 17,
 'bahut': 18,
 'tum': 19,
 'tha': 20,
 'log': 21,
 'best': 22,
 'happy': 23,
 'bhai': 24,
 'india': 25,
 'pakistan': 26,
 'jai': 27,
 'kuch': 28,
 'congress': 29,
 'par': 30,
 'mein': 31,
 'diya': 32,
 'liye': 33,
 'main': 34,
 'allah': 35,
 'apne': 36,
 'yeh': 37,
 'mai': 38,
 'khud': 39,
 'toh': 40,
 'one': 41,
 'har': 42,
 'rahe': 43,
 'bharat': 44,
 'gaya': 45,
 'hindu': 46,
 'ram': 47,
 'media': 48,
 'birthday': 49,
 'like': 50,
 'jaise': 51,
 'sab': 52,
 'aaj': 53,
 'logo': 54,
 'hum': 55,
 'world': 56,
 'chor': 57,
 'teri': 58,
 'maa': 59,
 'kisi': 60,
 'kiya': 61,
 'raha': 62,
 'day': 63,
 'jab': 64,
 'agar': 65,
 'baat': 66,
 'thank': 67,
 'hoga': 68,
 'great': 69,
 'tere': 70,
 'congratulations': 71,
 'jeet': 72,
 'dil': 73,
 'tak': 74,
 'hua': 75,
 'kabhi': 76,
 'muslim': 77,
 

In [40]:
train_len = train_texts.shape[0]
print (train_len)
trainX = X[:train_len]
trainy = y[:train_len]
valX = X[train_len:]
valy = y[train_len:]

14000


In [41]:
train_texts.text.iloc[1]

'min lyching manakgupta mein kahna nae chahta qki mere yaha btay tco jwsdvvomt8'

In [42]:
trainX[1]

array([ 5706,  9326,  2220,    31,  2221,  1128,   680,  2222,   111,
         362, 14901,   103, 14902,     0,     0,     0,     0,     0,
           0,     0], dtype=int32)

In [43]:
for word, i in word_index.items():
    if word not in embeddings_index1:
        print (word)

UNK


In [44]:
embedding_matrix1 = np.zeros((len(word_index) + 1, embed_dim1))
embedding_matrix2 = np.zeros((len(word_index) + 1, embed_dim2))

for word, i in word_index.items():
    if word in embeddings_index1:
        # words not found in embedding index will be all-zeros.
        embedding_vector = embeddings_index1.get(word)
        embedding_matrix1[i,:] = np.array(embedding_vector)
        
for word, i in word_index.items():    
    if word in embeddings_index2:
        # words not found in embedding index will be all-zeros.
        embedding_vector = embeddings_index2.get(word)
        embedding_matrix2[i,:] = np.array(embedding_vector)

In [45]:
print (embedding_matrix1.shape, embedding_matrix2.shape)

(16001, 200) (16001, 300)


In [46]:
import tensorflow as tf
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    #f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [47]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)), name='W')
        self.b = K.variable(self.init((self.attention_dim, )), name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)), name='u')
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [51]:
def get_model():
    input = Input(shape=(max_len,))
    emb1 = Embedding(len(word_index) + 1, embed_dim1, weights=[embedding_matrix1], trainable=True)(input)
    emb2 = Embedding(len(word_index) + 1, embed_dim2, weights=[embedding_matrix2], trainable=False)(input)
    #cnn = Conv1D(filters=100,kernel_size=5,strides=1,padding="same")(Concatenate(-1)([emb1,emb2]))
    #cnn = SpatialDropout1D(.2)(Concatenate(-1)([emb1,emb2]))
    out1 = Bidirectional(LSTM(lstm_out, dropout=0.2,return_sequences=True))(emb1)
    out2 = Bidirectional(LSTM(lstm_out//2, dropout=0.2, return_sequences=False))(out1)
    out1_attn = AttLayer(lstm_out//2)(out1)
    
    out3 = Bidirectional(LSTM(lstm_out, dropout=0.2,return_sequences=True))(emb2)
    out4 = Bidirectional(LSTM(lstm_out//2, dropout=0.2, return_sequences=False))(out3)
    out3_attn = AttLayer(lstm_out//2)(out1)
    
    out = Concatenate(-1)([out2,out1_attn,out4,out3_attn])
    out = Dropout(.2)(out)
    out = Dense(lstm_out)(out)
    out = Dropout(.2)(out)
    out = Dense(lstm_out//2)(out)
    out = Dropout(.2)(out)
    out = Dense(lstm_out//4)(out)
    out = Dropout(.2)(out)
    out = Dense(n_output,activation='softmax')(out)
    model = Model(input,out)
    model.compile(loss = 'categorical_crossentropy', optimizer=Adam(.0001),metrics = ['accuracy'])
    #print(model.summary())
    return model

In [52]:
model = get_model()
model.summary()

W0303 09:36:55.860455 4484668864 module_wrapper.py:139] From /Users/victor/Documents/nmt-keras/src/keras/keras/optimizers.py:1925: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0303 09:36:55.866902 4484668864 module_wrapper.py:139] From /Users/victor/Documents/nmt-keras/src/keras/keras/backend/tensorflow_backend.py:3827: The name tf.log is deprecated. Please use tf.math.log instead.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 20, 200)      3200200     input_2[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 20, 300)      4800300     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 20, 256)      336896      embedding_3[0][0]                
____________________________________________________________________________________________

In [53]:
#from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [54]:
batch_size = 32
n_epochs = 50

In [55]:
early = EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='max', baseline=None, restore_best_weights=False)
lr = ReduceLROnPlateau(monitor='val_acc', factor=0.7, patience=5, verbose=1, mode='auto', min_lr=0.000001)
checkpointer = ModelCheckpoint(monitor='val_acc',filepath='../models/weights_cmsa_wordlevel.hdf5', mode='max',verbose=1, save_best_only=True)

history = model.fit(trainX, trainy, epochs = n_epochs, batch_size=batch_size, verbose = 2, validation_data=(valX,valy), callbacks=[early,lr,checkpointer])

W0303 09:37:54.532935 4484668864 deprecation.py:323] From /Users/victor/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0303 09:37:58.794992 4484668864 module_wrapper.py:139] From /Users/victor/Documents/nmt-keras/src/keras/keras/backend/tensorflow_backend.py:1051: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0303 09:37:59.189319 4484668864 module_wrapper.py:139] From /Users/victor/Documents/nmt-keras/src/keras/keras/backend/tensorflow_backend.py:1038: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 14000 samples, validate on 3000 samples
Epoch 1/50
 - 70s - loss: 0.9311 - acc: 0.5388 - val_loss: 0.8804 - val_acc: 0.5850

Epoch 00001: val_acc improved from -inf to 0.58500, saving model to ../models/weights_cmsa_wordlevel.hdf5
Epoch 2/50
 - 57s - loss: 0.8842 - acc: 0.5803 - val_loss: 0.8626 - val_acc: 0.5943

Epoch 00002: val_acc improved from 0.58500 to 0.59433, saving model to ../models/weights_cmsa_wordlevel.hdf5
Epoch 3/50
 - 58s - loss: 0.8660 - acc: 0.5939 - val_loss: 0.8497 - val_acc: 0.6073

Epoch 00003: val_acc improved from 0.59433 to 0.60733, saving model to ../models/weights_cmsa_wordlevel.hdf5
Epoch 4/50
 - 62s - loss: 0.8379 - acc: 0.6142 - val_loss: 0.8408 - val_acc: 0.6120

Epoch 00004: val_acc improved from 0.60733 to 0.61200, saving model to ../models/weights_cmsa_wordlevel.hdf5
Epoch 5/50
 - 56s - loss: 0.8187 - acc: 0.6269 - val_loss: 0.8361 - val_acc: 0.6207

Epoch 00005: val_acc improved from 0.61200 to 0.62067, saving model to ../models/weights_cmsa

KeyboardInterrupt: 

In [96]:
from keras.callbacks import Callback

In [97]:
class Snapshotensemble(Callback):
    
    def __init__(self, valid_data, test_data, batch_size=16, fold=None):

        self.valid_inputs = valid_data[0]
        self.valid_outputs = valid_data[1]
        self.test_inputs = test_data
        self.best_value = 0
        self.batch_size = batch_size
        self.fold = fold
        
    def on_train_begin(self, logs={}):
        self.valid_predictions = []
        self.test_predictions = []
        
    def on_epoch_end(self, epoch, logs={}):
        self.valid_predictions.append(
            self.model.predict(self.valid_inputs, batch_size=self.batch_size))
        
        self.test_predictions.append(
            self.model.predict(self.test_inputs, batch_size=self.batch_size)
        )

In [89]:
def get_model_withemb():
    input = Input(shape=(max_len,))
    emb1 = Embedding(len(word_index) + 1, embed_dim1, weights=[embedding_matrix1], trainable=True)(input)
    emb2 = Embedding(len(word_index) + 1, embed_dim2, weights=[embedding_matrix2], trainable=False)(input)
    #cnn = Conv1D(filters=100,kernel_size=5,strides=1,padding="same")(Concatenate(-1)([emb1,emb2]))
    cnn = SpatialDropout1D(.2)(Concatenate(-1)([emb1,emb2]))
    out = Bidirectional(LSTM(lstm_out, dropout=0.2,return_sequences=True))(cnn)
    out = Bidirectional(LSTM(lstm_out//2, dropout=0.2, return_sequences=True))(out)
    out = AttLayer(lstm_out//2)(out)
    out = Dense(n_output,activation='softmax')(out)
    model = Model(input,out)
    model.compile(loss = 'categorical_crossentropy', optimizer=Adam(.0001),metrics = ['accuracy'])
    #print(model.summary())
    return model

In [90]:
model = get_model_withemb()
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 20, 200)      3200200     input_8[0][0]                    
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 20, 300)      4800300     input_8[0][0]                    
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 20, 500)      0           embedding_9[0][0]                
                                                                 embedding_10[0][0]         

In [99]:
early = EarlyStopping(monitor='val_acc', patience=20, verbose=1, mode='max', baseline=None, restore_best_weights=False)
lr = ReduceLROnPlateau(monitor='val_acc', factor=0.7, patience=8, verbose=1, mode='auto', min_lr=0.000001)
checkpointer = ModelCheckpoint(monitor='val_acc',filepath='../models/weights_finalmodel3.hdf5', mode='max',verbose=1, save_best_only=True)
snapshot = Snapshotensemble([valX,valy],testX)

history = model.fit(trainX, trainy, epochs = n_epochs, batch_size=batch_size, verbose = 2, validation_data=(valX,valy), callbacks=[early,lr,checkpointer,snapshot])


Train on 14000 samples, validate on 3000 samples
Epoch 1/50
 - 55s - loss: 0.6604 - acc: 0.7216 - val_loss: 0.8584 - val_acc: 0.6170

Epoch 00001: val_acc improved from -inf to 0.61700, saving model to ../models/weights_finalmodel3.hdf5
Epoch 2/50
 - 51s - loss: 0.6547 - acc: 0.7259 - val_loss: 0.8563 - val_acc: 0.6213

Epoch 00002: val_acc improved from 0.61700 to 0.62133, saving model to ../models/weights_finalmodel3.hdf5
Epoch 3/50
 - 54s - loss: 0.6477 - acc: 0.7251 - val_loss: 0.8628 - val_acc: 0.6213

Epoch 00003: val_acc did not improve from 0.62133
Epoch 4/50
 - 53s - loss: 0.6405 - acc: 0.7356 - val_loss: 0.8615 - val_acc: 0.6193

Epoch 00004: val_acc did not improve from 0.62133
Epoch 5/50
 - 54s - loss: 0.6414 - acc: 0.7294 - val_loss: 0.8667 - val_acc: 0.6223

Epoch 00005: val_acc improved from 0.62133 to 0.62233, saving model to ../models/weights_finalmodel3.hdf5
Epoch 6/50
 - 56s - loss: 0.6273 - acc: 0.7412 - val_loss: 0.8732 - val_acc: 0.6240

Epoch 00006: val_acc impro

In [100]:
model.load_weights('../models/weights_finalmodel3.hdf5')

In [103]:
np.array(snapshot.valid_predictions).shape

(50, 3000, 3)

In [107]:
np.array(snapshot.valid_predictions).mean(0).shape

(3000, 3)

In [118]:
val_pred = model.predict(valX).argmax(axis=1) #np.array(snapshot.valid_predictions[:5]).mean(0).argmax(axis=1)
val_texts['sentiment_pred'] = [in_senti_dict[i] for i in val_pred]
#val_texts = val_texts.sort_values(['uid'],ascending=[True]).reset_index(drop=True)

In [119]:
val_texts.head(5)

,uid,text,sentiment,sentiment_pred,sentiment_pred2,sentiment_pred3
0,3,pakistan ghra tauq pakistan israel tasleem nah...,negative,negative,positive,neutral
13,12,gonna start another june sour note uhhhh yes y...,neutral,positive,positive,positive
23,23,caring bohot jyada caring courier wale bsdk si...,negative,negative,negative,negative
35,24,sarfaraza nonesense kabhi baymani per bani tea...,positive,negative,neutral,neutral
49,45,pakistani team effort aagey allah marziiiiiiii...,positive,positive,neutral,positive


In [120]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

In [121]:
print (f1_score(val_texts.sentiment,val_texts.sentiment_pred,average='macro'))
print (accuracy_score(val_texts.sentiment,val_texts.sentiment_pred))

0.6286077414937643
0.6243333333333333


In [60]:
confusion_matrix(val_texts.sentiment,val_texts.sentiment_pred,labels=['negative','neutral','positive'])

array([[598, 224,  68],
       [274, 564, 290],
       [ 82, 209, 691]])

In [122]:
test_pred = np.array(snapshot.test_predictions).mean(0).argmax(1)
test_texts['sentiment_pred2'] = [in_senti_dict[i] for i in test_pred]

In [61]:
test_pred = model.predict(testX).argmax(axis=1)
test_texts['sentiment_pred'] = [in_senti_dict[i] for i in test_pred]
#test_texts_without_sort = test_texts.copy()
#test_texts = test_texts.sort_values(['uid'],ascending=[True]).reset_index(drop=True)

In [123]:
test_texts.head(10)

,uid,text,sentiment_pred,sentiment_pred3,sentiment_pred2
0,1,keh aese rahe jaise pakistan wale karte south ...,negative,negative,negative
1,4,anus prerna way ran saving ram ram jay rajaraa...,neutral,neutral,neutral
2,17,shukar hai pathan nae warna ptm nay wae dramay...,negative,neutral,neutral
3,18,harsh pen decision much option arm big second ...,positive,positive,positive
4,34,keep saying kenyan rugby beautiful save kru dr...,positive,neutral,positive
5,43,guru give bless sewa simrn parmarth good deeds...,positive,positive,positive
6,136,tum logo zindagi mai kabhi india map nahi dhek...,neutral,neutral,neutral
7,139,itne gandi priye aapke profile pic modi lga le...,neutral,neutral,neutral
8,158,google translate working aisi jagah net aata,neutral,neutral,neutral
9,162,qaira sahb great man apka beta nhayat nfees na...,neutral,positive,positive


In [63]:
test_texts.sentiment_pred.value_counts()

neutral     1079
positive    1000
negative     921
Name: sentiment_pred, dtype: int64

In [64]:
val_texts.sentiment_pred.value_counts()

positive    1049
neutral      997
negative     954
Name: sentiment_pred, dtype: int64

In [65]:
val_texts.sentiment.value_counts()

neutral     1128
positive     982
negative     890
Name: sentiment, dtype: int64

In [66]:
from LJST_script_BTM import *

minlabel = 0
maxlabel = 2
sentirange = maxlabel-minlabel
numwordspertopic=5
skipgramwindow=5

numsentilabel=3
numtopics=10
alpha=10
beta=.01
gamma=10.0
maxiter=10

unlabelled_data = pd.concat([val_texts[['text']],test_texts[['text']]],axis=0)

ljst_model = run_experiment(train_texts.text,trainy.argmax(1),unlabelled_data.text,sentirange,minlabel,maxlabel,numsentilabel,numtopics,skipgramwindow,alpha,beta,gamma,maxiter,numwordspertopic)


Starting iteration 1 of 10
Starting iteration 2 of 10
Starting iteration 3 of 10
Starting iteration 4 of 10
Starting iteration 5 of 10
Starting iteration 6 of 10
Starting iteration 7 of 10
Starting iteration 8 of 10
Starting iteration 9 of 10
Starting iteration 10 of 10
done in 1136.884s.


In [67]:
ds_estimated = []
for i in range(len(val_texts)):
    sentiment = 0
    index = len(train_texts) + i
    temp = np.matmul(ljst_model.dt_distribution[index,:],ljst_model.dts_distribution[index,:,:])
    sentiment = temp.argmax()
    #for k, val in enumerate(temp):
    #    sentiment += k*binsize*val
    ds_estimated.append(sentiment)

In [68]:
val_texts['sentiment_pred2'] = [in_senti_dict[i] for i in ds_estimated]

In [69]:
print (f1_score(val_texts.sentiment,val_texts.sentiment_pred2,average='macro'))
print (accuracy_score(val_texts.sentiment,val_texts.sentiment_pred2))

0.5071254291113468
0.5173333333333333


In [72]:
val_texts.head(50)

,uid,text,sentiment,sentiment_pred,sentiment_pred2
0,3,pakistan ghra tauq pakistan israel tasleem nah...,negative,negative,positive
13,12,gonna start another june sour note uhhhh yes y...,neutral,positive,positive
23,23,caring bohot jyada caring courier wale bsdk si...,negative,negative,negative
35,24,sarfaraza nonesense kabhi baymani per bani tea...,positive,negative,neutral
49,45,pakistani team effort aagey allah marziiiiiiii...,positive,positive,neutral
59,61,ankita shah8 bhadve photo elections pehle jyad...,negative,negative,negative
69,64,krishna jcb full trend chal rahi,positive,neutral,neutral
75,77,sharma1 kavita sharma4 sunita kal jab evm seal...,negative,negative,negative
90,81,accha kiya invite nai kiya corrupted party ind...,negative,negative,neutral
103,101,nolo weni ankere gae weekend,positive,neutral,neutral


In [81]:
def get_model_withtm():
    input1 = Input(shape=(max_len,))
    input2 = Input(shape=(numtopics,))
    #input3 = Input(shape=(numtopics*numsentilabel,))
    
    emb1 = Embedding(len(word_index) + 1, embed_dim1, weights=[embedding_matrix1], trainable=False)(input1)
    emb2 = Embedding(len(word_index) + 1, embed_dim2, weights=[embedding_matrix2], trainable=False)(input1)
    #cnn = Conv1D(filters=100,kernel_size=5,strides=1,padding="same")(Concatenate(-1)([emb1,emb2]))
    cnn = SpatialDropout1D(.2)(Concatenate(-1)([emb1,emb2]))
    out = Bidirectional(LSTM(lstm_out, dropout=0.2,return_sequences=True))(cnn)
    out = Bidirectional(LSTM(lstm_out//2, dropout=0.2, return_sequences=True))(out)
    out = AttLayer(lstm_out//2)(out)
    
    out = Concatenate(-1)([out,input2])
    
    out = Dense(n_output,activation='softmax')(out)
    model = Model([input1,input2],out)
    model.compile(loss = 'categorical_crossentropy', optimizer=Adam(.0005),metrics = ['accuracy'])
    #print(model.summary())
    return model

In [82]:
model = get_model_withtm()
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 20, 200)      3200200     input_6[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 20, 300)      4800300     input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 20, 500)      0           embedding_7[0][0]                
                                                                 embedding_8[0][0]          

In [79]:
train_tm1 = ljst_model.dt_distribution[:len(train_texts)]
val_tm1 = ljst_model.dt_distribution[len(train_texts):len(train_texts)+len(val_texts)]
test_tm1 = ljst_model.dt_distribution[len(train_texts)+len(val_texts):]

train_tm2 = ljst_model.dts_distribution[:len(train_texts)].reshape(len(train_texts),-1)
val_tm2 = ljst_model.dts_distribution[len(train_texts):len(train_texts)+len(val_texts)].reshape(len(val_texts),-1)
test_tm2 = ljst_model.dts_distribution[len(train_texts)+len(val_texts):].reshape(len(test_texts),-1)

In [83]:
early = EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='max', baseline=None, restore_best_weights=False)
lr = ReduceLROnPlateau(monitor='val_acc', factor=0.7, patience=5, verbose=1, mode='auto', min_lr=0.000001)
checkpointer = ModelCheckpoint(monitor='val_acc',filepath='../models/weights_finalmodel2.hdf5', mode='max',verbose=1, save_best_only=True)

history = model.fit([trainX,train_tm1], trainy, epochs = n_epochs, batch_size=batch_size, verbose = 2, validation_data=([valX,val_tm1],valy), callbacks=[early,lr,checkpointer])



Train on 14000 samples, validate on 3000 samples
Epoch 1/50
 - 32s - loss: 0.9028 - acc: 0.5614 - val_loss: 0.8747 - val_acc: 0.5880

Epoch 00001: val_acc improved from -inf to 0.58800, saving model to ../models/weights_finalmodel2.hdf5
Epoch 2/50
 - 27s - loss: 0.8697 - acc: 0.5826 - val_loss: 0.8643 - val_acc: 0.5830

Epoch 00002: val_acc did not improve from 0.58800
Epoch 3/50
 - 27s - loss: 0.8608 - acc: 0.5873 - val_loss: 0.8490 - val_acc: 0.6100

Epoch 00003: val_acc improved from 0.58800 to 0.61000, saving model to ../models/weights_finalmodel2.hdf5
Epoch 4/50
 - 27s - loss: 0.8511 - acc: 0.5930 - val_loss: 0.8403 - val_acc: 0.6083

Epoch 00004: val_acc did not improve from 0.61000
Epoch 5/50
 - 27s - loss: 0.8462 - acc: 0.5994 - val_loss: 0.8417 - val_acc: 0.6160

Epoch 00005: val_acc improved from 0.61000 to 0.61600, saving model to ../models/weights_finalmodel2.hdf5
Epoch 6/50
 - 31s - loss: 0.8378 - acc: 0.6056 - val_loss: 0.8290 - val_acc: 0.6157

Epoch 00006: val_acc did n

In [86]:
val_pred = model.predict([valX,val_tm1]).argmax(axis=1)
val_texts['sentiment_pred3'] = [in_senti_dict[i] for i in val_pred]

In [87]:
val_texts.head(50)

,uid,text,sentiment,sentiment_pred,sentiment_pred2,sentiment_pred3
0,3,pakistan ghra tauq pakistan israel tasleem nah...,negative,negative,positive,neutral
13,12,gonna start another june sour note uhhhh yes y...,neutral,positive,positive,positive
23,23,caring bohot jyada caring courier wale bsdk si...,negative,negative,negative,negative
35,24,sarfaraza nonesense kabhi baymani per bani tea...,positive,negative,neutral,neutral
49,45,pakistani team effort aagey allah marziiiiiiii...,positive,positive,neutral,positive
59,61,ankita shah8 bhadve photo elections pehle jyad...,negative,negative,negative,negative
69,64,krishna jcb full trend chal rahi,positive,neutral,neutral,neutral
75,77,sharma1 kavita sharma4 sunita kal jab evm seal...,negative,negative,negative,neutral
90,81,accha kiya invite nai kiya corrupted party ind...,negative,negative,neutral,negative
103,101,nolo weni ankere gae weekend,positive,neutral,neutral,neutral


In [88]:
print (f1_score(val_texts.sentiment,val_texts.sentiment_pred3,average='macro'))
print (accuracy_score(val_texts.sentiment,val_texts.sentiment_pred3))

0.6023244648780471
0.5973333333333334


In [190]:
val_texts[['uid','sentiment_pred']].to_csv('../answer.txt',header=['Uid','Sentiment'],index=False)

In [124]:
test_texts['sentiment'] = test_texts.sentiment_pred2.copy()

In [125]:
test_texts[['uid','sentiment']].to_csv('../answer.txt',index=None)